**결측치 채우는 방법**
- 만약 년도/도시를 기준으로 하면 test set에서 데이터 전처리를 진행할 때 test set의 데이터를 참고하게 될 수 밖에 없음
    - 각 년도/도시/라벨/인종/성별 별로 하나의 데이터 씩 밖에 없기 때문
- 년도와 다르게 도시는 가까운 도시를 정의할 방법이 필요
    - 혹은 U.S. Total의 값을 적용하는 방법이 있음
- 도시간 거리를 정의한다면
    - 각 (인종,성별,도시,연도) tuple 별로 각 라벨에서의 데이터 값에 대해서 sppt에서의 거리와 label value에서의 거리 사이에 비교
    - sppt 에서의 metric
        - 각각의 차원에서의 거리를 규정한 뒤, 각 차원에서의 거리를 euclidean 
        - 기타 방법이 가능할 수 있으나 데이터의 양을 봤을 때 쉽지 않을 듯
        - 인종, 성별 : all, both 와의 거리는 다른 집단과의 거리보다 짧게
        - 도시 : 도시에 대한 geo_strata 정보를 기준으로 metric 정의
            - geo_strata의 feature : region, poverty, population, popDensity, segration
            - 추가적으로 feature를 더한다면 : 내륙/해안/접경, 위도경도
            - geo_strata의 값이 연도 별로 달라지는 지 확인
        - 연도 : 차이

- 각 label 별로 예측 학습을 진행하여 전처리에 이용할 수도 있음
    - race-sex에 따른 차이에 경향성이 유사한 것들끼리 묶어서 하는 것 가능하면 의미 있을지도
    - category 혹은 sub category 별로 변화 사이에서 correlation을 보는 등
    - 혹은 특정 label에 대해서 학습을 진행한 뒤, 다른 label에 그대로 적용


In [1]:
import numpy as np 
import pandas as pd 
import matplotlib as mpl
import matplotlib.pyplot as plt 
import missingno as msno
import seaborn as sns 
import os

In [2]:
import matplotlib_inline.backend_inline

matplotlib_inline.backend_inline.set_matplotlib_formats("png2x")
# 테마 설정: "default", "classic", "dark_background", "fivethirtyeight", "seaborn"
mpl.style.use("fivethirtyeight")
# 이미지가 레이아웃 안으로 들어오도록 함
mpl.rcParams.update({"figure.constrained_layout.use": True})

In [3]:
import matplotlib.font_manager as fm
font_list = fm.findSystemFonts(fontpaths=None, fontext='ttf')
[fm.FontProperties(fname=font).get_name() for font in font_list if 'D2C' in font]
plt.rc('font', family='D2Coding')
mpl.rcParams['axes.unicode_minus'] = False

In [4]:
DATASET_DIR = '/home/doeun/code/AI/ESTSOFT2024/workspace/dataset/'
load_dir = 'america_big_cities_health_inventory'
file_name = 'BigCitiesHealth.csv'
RSLT_DIR = '/home/doeun/code/AI/ESTSOFT2024/workspace/1.project1_structured/BCHI/processed/'
pvtb_dir = RSLT_DIR + 'pvtb/'

## FUNCTIONS

- preprocess

In [5]:
def count_key_opt(data:pd.DataFrame,key,opt):
    rslt = dict()
    labels = data[key].unique()
    form = data[opt].value_counts().sort_values(ascending=False)
    form.iloc[:] = 0

    for feat in labels:
        cond = data[key]==feat
        val = form.copy()
        temp = data.loc[cond,opt].value_counts()
        val.loc[temp.index] = temp
        rslt[feat] = val

    return pd.DataFrame(rslt).T

In [6]:
import functools

def cond_check_dict(data=pd.DataFrame,val_dict=dict):
    cond_list=[
        data[col] == val
        for col, val in val_dict.items()
    ]
    return functools.reduce(lambda x,y: x & y, cond_list)

- plot

In [7]:
import re

def choose_split_point(word_len,space,ths):
    # 윗 줄에 space 만큼 공백이 있고, 한 줄의 길이가 ths로 제한 되어있을 때
    # 어떤 지점에서 단어를 끊어줄지 정하기
    # |-------ths-------|
    # |-space-|---------|-space-|------| : word
    #         |-------ths-------|
    print(word_len,space,ths)
    if word_len < ths + space :
        if abs(word_len/2 -ths) <= abs(word_len/2-space) :
            return word_len-ths
        else :
            return word_len - space if word_len < 2 * space else space
    else :
        return ths if word_len - (ths + space) < 0.3 * ths else space

def minimize_seq_idx_np(domain:np.array,func):
    vfunc = np.vectorize(func)
    temp = np.argsort(vfunc(domain))
    return temp[0]

def modify_strlen_ths(last,new,ths=16):
    front = len(last)
    space = ths - (1+front)
    if len(new) < space :
        rslt = [last + ' ' + new]
    else :
        if len(new) < ths:
            rslt = [last, new]
        else:
            cut = choose_split_point(len(new),space-1,ths-1)
            new_h, new_e = new[:cut]+'-', new[cut:]
            if cut < ths-1 :
                rslt = modify_strlen_ths(last+' '+new_h,new_e)
            else :
                rslt = [last] + modify_strlen_ths(new_h,new_e) 
    return rslt

def str_cutter(sentnc, ths = 16):
    words= sentnc.split(' ')
    rslt, pnt = [''], 0
    while pnt < len(words):
        last = '' if len(rslt)==0 else rslt[-1]
        next_ele = modify_strlen_ths(last,words[pnt],ths)
        rslt = rslt[:-1] + next_ele
        pnt += 1
    return '\n'.join(rslt)[1:]
#알고리즘 때문에 맨 앞에 빈칸 하나 들어가게 되는 이슈 있음

print(str_cutter('Racial Segregation Indices | Racial Segregation, White and Hispanic', 13))



Racial
Segregation
Indices |
Racial
Segregation,
White and
Hispanic


In [8]:
def choose_plot_grid(n:int,r_max=8,c_max=17):
    rs = np.sqrt(n)
    r_min = np.ceil(n/c_max)
    sppt = np.arange(r_min,r_max+1) #need error process
    col_nums = np.ceil(n/sppt)
    res = col_nums * sppt -n
    min_idx = np.where(res==np.min(res))
    #TODO : if len(min_idx) > 1 : minimize abs(sppt-col_nums), col_nums
    return sppt[min_idx[0]], col_nums[min_idx[0]]


def plot_feat_hue(data:pd.DataFrame,hue_label_dict=None):
    num_r, num_c = choose_plot_grid(len(data))
    fig, axes = plt.subplots(num_r,num_c,figsize=(21,17),sharex=True,sharey=True)
    for n,ax in enumerate(axes.flatten()):
        plt.setp(ax.get_xticklabels(),ha = 'left',rotation = 90)
        if n >= len(data) : continue
        sns.barplot(x=data.iloc[n].index, y = data.iloc[n].values,ax =ax)
        feat_name = data.index[n]
        if hue_label_dict: color = 'b' if hue_label_dict[feat_name] else 'k'
        else : color = 'k'
        ax.set_xlabel(str_cutter(feat_name,20),loc='left',fontsize = 8.3,color=color)

    pass

## LOAD DATA

In [9]:
pvtb_name = 'pvtb_city_entire_ver0.csv'


pvtb_path = os.path.join(pvtb_dir,pvtb_name)
pvtb_entire = pd.read_csv(pvtb_path)

pvtb_entire

,geo_label_city,date_label,strata_race_label,strata_sex_label,Active Transportation | Riding Bike to Work,Active Transportation | Walking to Work,Air Pollution | Hazardous Air Quality,Air Pollution | Poor Air Quality,Birth Control | Teen Birth Control,Births | Low Birthweight,...,Substance Use | Adult Smoking,Substance Use | Drug Overdose Deaths,Substance Use | Opioid Overdose Deaths,Substance Use | Teen Alcohol,Substance Use | Teen Marijuana,Substance Use | Teen Smoking,Transportation | Drives Alone to Work,Transportation | Lack of Car,Transportation | Longer Driving Commute Time,Transportation | Public Transportation Use
0,Austin,2010,All,Both,1.458997,2.506039,2.191781,25.479452,NaN,7.836414,...,14.1,14.814379,5.316296,NaN,NaN,NaN,71.701852,7.104797,NaN,4.673879
1,Austin,2010,All,Female,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,12.110401,4.224056,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Austin,2010,All,Male,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,17.451652,6.417004,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Austin,2010,Asian/PI,Both,NaN,NaN,NaN,NaN,NaN,8.785406,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Austin,2010,Asian/PI,Female,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7275,Washington,2022,Hispanic,Male,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,25.584642,21.067009,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7276,Washington,2022,Natives,Both,NaN,8.766234,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,19.237013,NaN,NaN,38.879870
7277,Washington,2022,White,Both,NaN,14.201808,NaN,NaN,NaN,5.574537,...,NaN,13.498547,8.887272,NaN,NaN,NaN,24.173919,NaN,NaN,21.684225
7278,Washington,2022,White,Female,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,6.485127,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Define metic on a support $X$

- 숫자들 간에는 차의 절대값으로 하면 됨
- categorical의 경우
    - geo : ordinal이라 볼 수 있지만 binary
    - race, sex : nominal이지만 중간값(all or both) 존재

In [10]:
raw_file_path = os.path.join(DATASET_DIR,load_dir,file_name)
df_raw = pd.read_csv(raw_file_path)

df_raw

/tmp/ipykernel_1957/1273264358.py:2: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.
  df_raw = pd.read_csv(raw_file_path)


,metric_item_label,metric_cat_label,metric_subcat_label,metric_item_label_subtitle,metric_cat_item_yaxis_label,metric_source_desc_label_fn,metric_source_desc_label_url_fn,geo_label_city,geo_label_state,geo_label_citystate,...,value_90_ci_low,value_90_ci_high,geo_strata_region,geo_strata_poverty,geo_strata_Population,geo_strata_PopDensity,geo_strata_Segregation,strata_race_label,strata_sex_label,strata_race_sex_label
0,All Cancer Deaths,Chronic Health Conditions,Cancer,"Deaths from all types of cancer (per 100,000 p...","Per 100,000","National Vital Statistics System (NVSS), Cente...",https://www.cdc.gov/nchs/nvss/index.htm,Cleveland,OH,"Cleveland, OH",...,NaN,NaN,Midwest,Poorest cities (18%+ poor),Smaller (<1.3 million),Lower pop. density (<10k per sq mi),Highly Segregated (50%+),Black,Female,Black Female
1,All Cancer Deaths,Chronic Health Conditions,Cancer,"Deaths from all types of cancer (per 100,000 p...","Per 100,000","National Vital Statistics System (NVSS), Cente...",https://www.cdc.gov/nchs/nvss/index.htm,Oakland,CA,"Oakland, CA",...,NaN,NaN,West,Less poor cities (<18% poor),Smaller (<1.3 million),Lower pop. density (<10k per sq mi),Less Segregated (<50%),Hispanic,Male,Hispanic Male
2,All Cancer Deaths,Chronic Health Conditions,Cancer,"Deaths from all types of cancer (per 100,000 p...","Per 100,000","National Vital Statistics System (NVSS), Cente...",https://www.cdc.gov/nchs/nvss/index.htm,San Francisco,CA,"San Francisco, CA",...,NaN,NaN,West,Less poor cities (<18% poor),Smaller (<1.3 million),Highest pop. density (>10k per sq mi),Less Segregated (<50%),All,Female,NaN
3,All Cancer Deaths,Chronic Health Conditions,Cancer,"Deaths from all types of cancer (per 100,000 p...","Per 100,000","National Vital Statistics System (NVSS), Cente...",https://www.cdc.gov/nchs/nvss/index.htm,Long Beach,CA,"Long Beach, CA",...,NaN,NaN,West,Less poor cities (<18% poor),Smaller (<1.3 million),Smaller (<1.3 million),Highly Segregated (50%+),Hispanic,Female,Hispanic Female
4,All Cancer Deaths,Chronic Health Conditions,Cancer,"Deaths from all types of cancer (per 100,000 p...","Per 100,000","National Vital Statistics System (NVSS), Cente...",https://www.cdc.gov/nchs/nvss/index.htm,Cleveland,OH,"Cleveland, OH",...,NaN,NaN,Midwest,Poorest cities (18%+ poor),Smaller (<1.3 million),Lower pop. density (<10k per sq mi),Highly Segregated (50%+),Black,Male,Black Male
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189974,"Racial Segregation, White and Hispanic",Social and Economic Factors,Racial Segregation Indices,Residential segregation of white non-Hispanic ...,Lower is better,"American Community Survey, U.S. Census Bureau",https://www.census.gov/programs-surveys/acs,Chicago,IL,"Chicago, IL",...,NaN,NaN,Midwest,Less poor cities (<18% poor),Largest (>1.3 million),Highest pop. density (>10k per sq mi),Highly Segregated (50%+),All,Both,NaN
189975,"Racial Segregation, White and Hispanic",Social and Economic Factors,Racial Segregation Indices,Residential segregation of white non-Hispanic ...,Lower is better,"American Community Survey, U.S. Census Bureau",https://www.census.gov/programs-surveys/acs,Charlotte,NC,"Charlotte, NC",...,NaN,NaN,South,Less poor cities (<18% poor),Smaller (<1.3 million),Lower pop. density (<10k per sq mi),Less Segregated (<50%),All,Both,NaN
189976,"Racial Segregation, White and Hispanic",Social and Economic Factors,Racial Segregation Indices,Residential segregation of white non-Hispanic ...,Lower is better,"American Community Survey, U.S. Census Bureau",https://www.census.gov/programs-surveys/acs,Boston,MA,"Boston, MA",...,NaN,NaN,Northeast,Less poor cities (<18% poor),Smaller (<1.3 million),Highest pop. density (>10k per sq mi),Highly Segregated (50%+),All,Both,NaN
189977,"Racial Segregation, White and Hispanic",Social and Economic Factors,Racial Segregation Indices,Residential segregation of white non-Hispanic ...,Lower is better,"American Community Survey, U.S. Census Bureau",https://w

In [11]:
cond_US = cond_check_dict(df_raw,{"geo_label_city":"U.S. Total"})
df_city = df_raw[~cond_US]

df_city

,metric_item_label,metric_cat_label,metric_subcat_label,metric_item_label_subtitle,metric_cat_item_yaxis_label,metric_source_desc_label_fn,metric_source_desc_label_url_fn,geo_label_city,geo_label_state,geo_label_citystate,...,value_90_ci_low,value_90_ci_high,geo_strata_region,geo_strata_poverty,geo_strata_Population,geo_strata_PopDensity,geo_strata_Segregation,strata_race_label,strata_sex_label,strata_race_sex_label
0,All Cancer Deaths,Chronic Health Conditions,Cancer,"Deaths from all types of cancer (per 100,000 p...","Per 100,000","National Vital Statistics System (NVSS), Cente...",https://www.cdc.gov/nchs/nvss/index.htm,Cleveland,OH,"Cleveland, OH",...,NaN,NaN,Midwest,Poorest cities (18%+ poor),Smaller (<1.3 million),Lower pop. density (<10k per sq mi),Highly Segregated (50%+),Black,Female,Black Female
1,All Cancer Deaths,Chronic Health Conditions,Cancer,"Deaths from all types of cancer (per 100,000 p...","Per 100,000","National Vital Statistics System (NVSS), Cente...",https://www.cdc.gov/nchs/nvss/index.htm,Oakland,CA,"Oakland, CA",...,NaN,NaN,West,Less poor cities (<18% poor),Smaller (<1.3 million),Lower pop. density (<10k per sq mi),Less Segregated (<50%),Hispanic,Male,Hispanic Male
2,All Cancer Deaths,Chronic Health Conditions,Cancer,"Deaths from all types of cancer (per 100,000 p...","Per 100,000","National Vital Statistics System (NVSS), Cente...",https://www.cdc.gov/nchs/nvss/index.htm,San Francisco,CA,"San Francisco, CA",...,NaN,NaN,West,Less poor cities (<18% poor),Smaller (<1.3 million),Highest pop. density (>10k per sq mi),Less Segregated (<50%),All,Female,NaN
3,All Cancer Deaths,Chronic Health Conditions,Cancer,"Deaths from all types of cancer (per 100,000 p...","Per 100,000","National Vital Statistics System (NVSS), Cente...",https://www.cdc.gov/nchs/nvss/index.htm,Long Beach,CA,"Long Beach, CA",...,NaN,NaN,West,Less poor cities (<18% poor),Smaller (<1.3 million),Smaller (<1.3 million),Highly Segregated (50%+),Hispanic,Female,Hispanic Female
4,All Cancer Deaths,Chronic Health Conditions,Cancer,"Deaths from all types of cancer (per 100,000 p...","Per 100,000","National Vital Statistics System (NVSS), Cente...",https://www.cdc.gov/nchs/nvss/index.htm,Cleveland,OH,"Cleveland, OH",...,NaN,NaN,Midwest,Poorest cities (18%+ poor),Smaller (<1.3 million),Lower pop. density (<10k per sq mi),Highly Segregated (50%+),Black,Male,Black Male
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189974,"Racial Segregation, White and Hispanic",Social and Economic Factors,Racial Segregation Indices,Residential segregation of white non-Hispanic ...,Lower is better,"American Community Survey, U.S. Census Bureau",https://www.census.gov/programs-surveys/acs,Chicago,IL,"Chicago, IL",...,NaN,NaN,Midwest,Less poor cities (<18% poor),Largest (>1.3 million),Highest pop. density (>10k per sq mi),Highly Segregated (50%+),All,Both,NaN
189975,"Racial Segregation, White and Hispanic",Social and Economic Factors,Racial Segregation Indices,Residential segregation of white non-Hispanic ...,Lower is better,"American Community Survey, U.S. Census Bureau",https://www.census.gov/programs-surveys/acs,Charlotte,NC,"Charlotte, NC",...,NaN,NaN,South,Less poor cities (<18% poor),Smaller (<1.3 million),Lower pop. density (<10k per sq mi),Less Segregated (<50%),All,Both,NaN
189976,"Racial Segregation, White and Hispanic",Social and Economic Factors,Racial Segregation Indices,Residential segregation of white non-Hispanic ...,Lower is better,"American Community Survey, U.S. Census Bureau",https://www.census.gov/programs-surveys/acs,Boston,MA,"Boston, MA",...,NaN,NaN,Northeast,Less poor cities (<18% poor),Smaller (<1.3 million),Highest pop. density (>10k per sq mi),Highly Segregated (50%+),All,Both,NaN
189977,"Racial Segregation, White and Hispanic",Social and Economic Factors,Racial Segregation Indices,Residential segregation of white non-Hispanic ...,Lower is better,"American Community Survey, U.S. Census Bureau",https://w

### metric on geo_strat
- abstract geo_strat

In [12]:
geo_strat_cols = list(filter(lambda x : 'geo_strat' in x,df_city.columns))
geo_info_cols = list(filter(lambda x : 'geo_label' in x,df_city.columns))[:3]

df_temp = df_city[geo_info_cols+geo_strat_cols]

df_temp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 188667 entries, 0 to 189978
Data columns (total 8 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   geo_label_city          188667 non-null  object
 1   geo_label_state         188667 non-null  object
 2   geo_label_citystate     188667 non-null  object
 3   geo_strata_region       188667 non-null  object
 4   geo_strata_poverty      188667 non-null  object
 5   geo_strata_Population   188666 non-null  object
 6   geo_strata_PopDensity   188667 non-null  object
 7   geo_strata_Segregation  188667 non-null  object
dtypes: object(8)
memory usage: 13.0+ MB


In [13]:
# EDA 결과, las vegas와 long beach에 대해서 오기입된 record가 하나씩 있음
# 오기입된 outlier을 제외하고 반복되어 등장한 것만 추림
dup_cond = df_temp.duplicated(keep=False)
print(len(df_temp[dup_cond].value_counts()))
#df_temp[dup_cond].value_counts()

35


In [71]:
df_geo_strat = df_temp[dup_cond].value_counts().reset_index()

In [72]:
df_geo_info = df_geo_strat.set_index('geo_label_city')

df_geo_info

,geo_label_state,geo_label_citystate,geo_strata_region,geo_strata_poverty,geo_strata_Population,geo_strata_PopDensity,geo_strata_Segregation,count
geo_label_city,,,,,,,,
New York City,NY,"New York City, NY",Northeast,Less poor cities (<18% poor),Largest (>1.3 million),Highest pop. density (>10k per sq mi),Highly Segregated (50%+),6535
Houston,TX,"Houston, TX",South,Poorest cities (18%+ poor),Largest (>1.3 million),Lower pop. density (<10k per sq mi),Highly Segregated (50%+),6431
Los Angeles,CA,"Los Angeles, CA",West,Less poor cities (<18% poor),Largest (>1.3 million),Lower pop. density (<10k per sq mi),Highly Segregated (50%+),6372
Philadelphia,PA,"Philadelphia, PA",Northeast,Poorest cities (18%+ poor),Largest (>1.3 million),Highest pop. density (>10k per sq mi),Highly Segregated (50%+),6310
San Diego,CA,"San Diego, CA",West,Less poor cities (<18% poor),Largest (>1.3 million),Lower pop. density (<10k per sq mi),Less Segregated (<50%),6251
Chicago,IL,"Chicago, IL",Midwest,Less poor cities (<18% poor),Largest (>1.3 million),Highest pop. density (>10k per sq mi),Highly Segregated (50%+),6210
San Francisco,CA,"San Francisco, CA",West,Less poor cities (<18% poor),Smaller (<1.3 million),Highest pop. density (>10k per sq mi),Less Segregated (<50%),6121
Las Vegas,NV,"Las Vegas, NV",West,Less poor cities (<18% poor),Smaller (<1.3 million),Lower pop. density (<10k per sq mi),Less Segregated (<50%),5848
Boston,MA,"Boston, MA",Northeast,Less poor cities (<18% poor),Smaller (<1.3 million),Highest pop. density (>10k per sq mi),Highly Segregated (50%+),5718


In [73]:
save_path = os.path.join(RSLT_DIR,'geo_strat_info.csv')
df_geo_info.to_csv(save_path,index=True)

- except for geo_strata_region, all of them is ordinal and binary
- in geo_strata_region, minimal length between each regions could be base of metric
    - a condition of metric :

        (distance between a, c) <= (distance between a, b) + (distance between b, c)

In [16]:
df_region = df_geo_strat.groupby('geo_strata_region')['geo_label_state'].apply(set)

df_region

geo_strata_region
Midwest      {OH, MO, WI, MN, IN, MI, IL}
Northeast                    {MA, NY, PA}
South        {DC, MD, TN, KY, TX, NC, OK}
West             {OR, NV, CA, CO, AZ, WA}
Name: geo_label_state, dtype: object

In [17]:
df_geo_strat[geo_strat_cols].value_counts()

geo_strata_region  geo_strata_poverty            geo_strata_Population   geo_strata_PopDensity                  geo_strata_Segregation  
West               Less poor cities (<18% poor)  Smaller (<1.3 million)  Lower pop. density (<10k per sq mi)    Less Segregated (<50%)      6
South              Less poor cities (<18% poor)  Smaller (<1.3 million)  Lower pop. density (<10k per sq mi)    Less Segregated (<50%)      5
Midwest            Less poor cities (<18% poor)  Smaller (<1.3 million)  Lower pop. density (<10k per sq mi)    Less Segregated (<50%)      4
                   Poorest cities (18%+ poor)    Smaller (<1.3 million)  Lower pop. density (<10k per sq mi)    Highly Segregated (50%+)    3
South              Poorest cities (18%+ poor)    Smaller (<1.3 million)  Lower pop. density (<10k per sq mi)    Highly Segregated (50%+)    2
West               Less poor cities (<18% poor)  Largest (>1.3 million)  Lower pop. density (<10k per sq mi)    Highly Segregated (50%+)    2
           

In [18]:
for col in geo_strat_cols:
    display(df_geo_strat[col].value_counts())

geo_strata_region
South        12
West         12
Midwest       8
Northeast     3
Name: count, dtype: int64

geo_strata_poverty
Less poor cities (<18% poor)    26
Poorest cities (18%+ poor)       9
Name: count, dtype: int64

geo_strata_Population
Smaller (<1.3 million)    26
Largest (>1.3 million)     9
Name: count, dtype: int64

geo_strata_PopDensity
Lower pop. density (<10k per sq mi)      29
Highest pop. density (>10k per sq mi)     6
Name: count, dtype: int64

geo_strata_Segregation
Less Segregated (<50%)      20
Highly Segregated (50%+)    15
Name: count, dtype: int64

basically,
- if a,b are adjacent : d(a,b) = 1
- if minimal path == 2 : d(a,b) = 1.732 (cf. sqrt 3)

(cf : 정삼각형 2 개 붙어있는 모양)

In [19]:
import itertools

region_adjacent = [
    { "Midwest", "Northeast" },
    { "Midwest", "South" },
    { "Midwest", "West" },
    { "West", "South" },
    { "South", "Northeast" }
]

    
def metric_on_adj(points,adjacents,far_distance = 1.732):
    metric_dict=dict()
    for a,b in itertools.product(points,repeat=2):
        if a == b : metric_dict[(a,b)] = 0
        elif {a,b} in adjacents : metric_dict[(a,b)] = 1
        else : metric_dict[(a,b)] = far_distance 
    return lambda x,y : metric_dict[(x,y)]

metric_region = metric_on_adj(df_region.index,region_adjacent)

print(type(metric_region))
print(metric_region('West','Northeast'))

<class 'function'>
1.732


the algorithm could be applied on other nominal categories, e.g. race, sex, and other binay ordinal categories.

In [20]:
metric_binary = lambda x,y : 0 if x==y else 1
# metric_binary.__setattr__('name','metric_binary') -> 함수에 이름 정해주는 법 어떻게?

entire_race = ['All','White','Black','Hispanic','Asian/PI','Natives']
race_adjacnet = [
    {'All', a}
    for a in entire_race
    if a != 'All'
]
metric_race = metric_on_adj(entire_race, race_adjacnet)

entire_sex = ['Both','Female','Male']
sex_adjacnet = [
    {'Both', a}
    for a in entire_race
    if a != 'Both'
]
metric_sex = metric_on_adj(entire_sex, sex_adjacnet)

metric_race('White','Black')

1.732

In [21]:
dict_metric = dict()

dict_metric['strata_race_label'] = metric_race
dict_metric['strata_sex_label'] = metric_sex
dict_metric['geo_strata_region'] = metric_region

for col in geo_strat_cols:
    if col == 'geo_strata_region' : continue
    dict_metric[col] = metric_binary

dict_metric

{'strata_race_label': <function __main__.metric_on_adj.<locals>.<lambda>(x, y)>,
 'strata_sex_label': <function __main__.metric_on_adj.<locals>.<lambda>(x, y)>,
 'geo_strata_region': <function __main__.metric_on_adj.<locals>.<lambda>(x, y)>,
 'geo_strata_poverty': <function __main__.<lambda>(x, y)>,
 'geo_strata_Population': <function __main__.<lambda>(x, y)>,
 'geo_strata_PopDensity': <function __main__.<lambda>(x, y)>,
 'geo_strata_Segregation': <function __main__.<lambda>(x, y)>}

In [22]:
dict_metric['geo_strata_Segregation'].__name__

'<lambda>'

In [23]:
dict_metric['geo_strata_region'] ('Midwest','Midwest')

0

In [24]:
dict_metric['strata_race_label'] ('All','White')

1

### distance between cities
- 각 axis 에서의 거리를 L7 norm 한 것으로 테스트
- 7은 임의로 정한 것으로 필요시 다른 값으로 테스트 해봐도 좋을 듯

In [74]:
df_geo_info

,geo_label_state,geo_label_citystate,geo_strata_region,geo_strata_poverty,geo_strata_Population,geo_strata_PopDensity,geo_strata_Segregation,count
geo_label_city,,,,,,,,
New York City,NY,"New York City, NY",Northeast,Less poor cities (<18% poor),Largest (>1.3 million),Highest pop. density (>10k per sq mi),Highly Segregated (50%+),6535
Houston,TX,"Houston, TX",South,Poorest cities (18%+ poor),Largest (>1.3 million),Lower pop. density (<10k per sq mi),Highly Segregated (50%+),6431
Los Angeles,CA,"Los Angeles, CA",West,Less poor cities (<18% poor),Largest (>1.3 million),Lower pop. density (<10k per sq mi),Highly Segregated (50%+),6372
Philadelphia,PA,"Philadelphia, PA",Northeast,Poorest cities (18%+ poor),Largest (>1.3 million),Highest pop. density (>10k per sq mi),Highly Segregated (50%+),6310
San Diego,CA,"San Diego, CA",West,Less poor cities (<18% poor),Largest (>1.3 million),Lower pop. density (<10k per sq mi),Less Segregated (<50%),6251
Chicago,IL,"Chicago, IL",Midwest,Less poor cities (<18% poor),Largest (>1.3 million),Highest pop. density (>10k per sq mi),Highly Segregated (50%+),6210
San Francisco,CA,"San Francisco, CA",West,Less poor cities (<18% poor),Smaller (<1.3 million),Highest pop. density (>10k per sq mi),Less Segregated (<50%),6121
Las Vegas,NV,"Las Vegas, NV",West,Less poor cities (<18% poor),Smaller (<1.3 million),Lower pop. density (<10k per sq mi),Less Segregated (<50%),5848
Boston,MA,"Boston, MA",Northeast,Less poor cities (<18% poor),Smaller (<1.3 million),Highest pop. density (>10k per sq mi),Highly Segregated (50%+),5718


In [26]:
print(metric_region('Northeast', 'Northeast'))

0


In [27]:
print(dict_metric['geo_strata_region']('Northeast', 'Northeast'))

0


In [44]:
city_list = list(df_geo_info.index)

metric_val= dict()
for a, b in itertools.product(city_list,repeat=2):
    diff = [
        dict_metric[col](df_geo_info.loc[a,col],
                         df_geo_info.loc[b,col])
        for col in geo_strat_cols
        ]
    metric_val[(a,b)] = np.linalg.norm(np.array(diff),ord=7)
    
metric_val 

{('New York City', 'New York City'): 0.0,
 ('New York City', 'Houston'): 1.169930812758687,
 ('New York City', 'Los Angeles'): 1.7372440623952268,
 ('New York City', 'Philadelphia'): 1.0,
 ('New York City', 'San Diego'): 1.7423948319061235,
 ('New York City', 'Chicago'): 1.0,
 ('New York City', 'San Francisco'): 1.7423948319061235,
 ('New York City', 'Las Vegas'): 1.7474558315662452,
 ('New York City', 'Boston'): 1.0,
 ('New York City', 'Oakland'): 1.7474558315662452,
 ('New York City', 'Dallas'): 1.1040895136738123,
 ('New York City', 'Fort Worth'): 1.2190136542044754,
 ('New York City', 'Seattle'): 1.7474558315662452,
 ('New York City', 'Phoenix'): 1.7372440623952268,
 ('New York City', 'Charlotte'): 1.2190136542044754,
 ('New York City', 'San Antonio'): 1.169930812758687,
 ('New York City', 'Washington'): 1.1040895136738123,
 ('New York City', 'Portland'): 1.7474558315662452,
 ('New York City', 'Long Beach'): 1.7423948319061235,
 ('New York City', 'San Jose'): 1.7474558315662452,
 (

In [52]:
metric_val[('Portland','Seattle')]

0.0

In [53]:
metric_val[('New York City', 'Denver')]

1.7474558315662452

In [37]:
temp = pd.Series(metric_val).reset_index()

pd.pivot_table(temp, index = 'level_0', columns = 'level_1')

0                                                    \
level_1          Austin Baltimore    Boston Charlotte   Chicago Cleveland   
level_0                                                                     
Austin         0.000000  1.104090  1.169931  0.000000  1.219014  1.169931   
Baltimore      1.104090  0.000000  1.169931  1.104090  1.219014  1.000000   
Boston         1.169931  1.169931  0.000000  1.169931  1.104090  1.169931   
Charlotte      0.000000  1.104090  1.169931  0.000000  1.219014  1.169931   
Chicago        1.219014  1.219014  1.104090  1.219014  0.000000  1.169931   
Cleveland      1.169931  1.000000  1.169931  1.169931  1.169931  0.000000   
Columbus       1.000000  1.169931  1.169931  1.000000  1.169931  1.104090   
Dallas         1.104090  1.104090  1.169931  1.104090  1.104090  1.169931   
Denver         1.000000  1.169931  1.742395  1.000000  1.219014  1.169931   
Detroit        1.169931  1.000000  1.169931  1.169931  1.169931  0.000000   
El Paso        1.000000  1.000000  1.219014  1.000000  1.258499  1.104090   
Fort Worth     0.000000  1.104090  1.169931  0.000000  1.219014  1.169931   
Houston        1.169931  1.000000  1.219014  1.169931  1.169931  1.104090   
Indianapolis   1.000000  1.169931  1.169931  1.000000  1.169931  1.104090   
Kansas City    1.000000  1.169931  1.169931  1.000000  1.169931  1.104090   
Las Vegas      1.000000  1.169931  1.742395  1.000000  1.219014  1.169931   
Long Beach     1.104090  1.104090  1.737244  1.104090  1.169931  1.104090   
Los Angeles    1.169931  1.169931  1.742395  1.169931  1.104090  1.169931   
Louisville     0.000000  1.104090  1.169931  0.000000  1.219014  1.169931   
Memphis        1.104090  0.000000  1.169931  1.104090  1.219014  1.000000   
Milwaukee      1.169931  1.000000  1.169931  1.169931  1.169931  0.000000   
Minneapolis    1.000000  1.169931  1.169931  1.000000  1.169931  1.104090   
New York City  1.219014  1.219014  1.000000  1.219014  1.000000  1.219014   
Oakland        1.000000  1.169931  1.742395  1.000000  1.219014  1.169931   
Oklahoma City  0.000000  1.104090  1.169931  0.000000  1.219014  1.169931   
Philadelphia   1.258499  1.169931  1.104090  1.258499  1.104090  1.169931   
Phoenix        1.169931  1.169931  1.742395  1.169931  1.104090  1.169931   
Portland       1.000000  1.169931  1.742395  1.000000  1.219014  1.169931   
San Antonio    1.000000  1.169931  1.219014  1.000000  1.169931  1.219014   
San Diego      1.104090  1.219014  1.747456  1.104090  1.169931  1.219014   
San Francisco  1.104090  1.219014  1.737244  1.104090  1.169931  1.219014   
San Jose       1.000000  1.169931  1.742395  1.000000  1.219014  1.169931   
Seattle        1.000000  1.169931  1.742395  1.000000  1.219014  1.169931   
Tucson         1.104090  1.104090  1.747456  1.104090  1.258499  1.104090   
Washington     1.104090  1.104090  1.000000  1.104090  1.104090  1.169931   

                                                       ...               \
level_1        Columbus    Dallas    Denver   Detroit  ... Philadelphia   
level_0                                                ...                
Austin         1.000000  1.104090  1.000000  1.169931  ...     1.258499   
Baltimore      1.169931  1.104090  1.169931  1.000000  ...     1.169931   
Boston         1.169931  1.169931  1.742395  1.169931  ...     1.104090   
Charlotte      1.000000  1.104090  1.000000  1.169931  ...     1.258499   
Chicago        1.169931  1.104090  1.219014  1.169931  ...     1.104090   
Cleveland      1.104090  1.169931  1.169931  0.000000  ...     1.169931   
Columbus       0.000000  1.169931  1.000000  1.104090  ...     1.258499   
Dallas         1.169931  0.000000  1.169931  1.169931  ...     1.169931   
Denver         1.000000  1.169931  0.000000  1.169931  ...     1.752430   
Detroit        1.104090  1.169931  1.169931  0.000000  ...     1.169931   
El Paso        1.104090  1.169931  1.104090  1.104090  ...     1.219014   
Fort Worth     1.000000  1.104090  1.000000  

In [64]:
#dict_metric['geo_label_city'] = lambda x,y : np.linalg.norm(np.array([metric_val[(x,y)],0.3 * metric_binary(x,y)]),ord=3)
dict_metric['geo_label_city'] = lambda x,y : metric_val[(x,y)]

In [65]:
dict_metric

{'strata_race_label': <function __main__.metric_on_adj.<locals>.<lambda>(x, y)>,
 'strata_sex_label': <function __main__.metric_on_adj.<locals>.<lambda>(x, y)>,
 'geo_strata_region': <function __main__.metric_on_adj.<locals>.<lambda>(x, y)>,
 'geo_strata_poverty': <function __main__.<lambda>(x, y)>,
 'geo_strata_Population': <function __main__.<lambda>(x, y)>,
 'geo_strata_PopDensity': <function __main__.<lambda>(x, y)>,
 'geo_strata_Segregation': <function __main__.<lambda>(x, y)>,
 'geo_label_city': <function __main__.<lambda>(x, y)>}

In [66]:
dict_metric['geo_label_city'] ('Portland','Seattle')

0.0

In [67]:
metric_val[('New York City', 'Denver')]

1.7474558315662452

In [68]:
metric_val[('Portland','Seattle')]

0.0

In [69]:
df_geo_info.loc[['Portland','Seattle']]

,geo_label_state,geo_label_citystate,geo_strata_region,geo_strata_poverty,geo_strata_Population,geo_strata_PopDensity,geo_strata_Segregation,count
geo_label_city,,,,,,,,
Portland,OR,"Portland, OR",West,Less poor cities (<18% poor),Smaller (<1.3 million),Lower pop. density (<10k per sq mi),Less Segregated (<50%),5219
Seattle,WA,"Seattle, WA",West,Less poor cities (<18% poor),Smaller (<1.3 million),Lower pop. density (<10k per sq mi),Less Segregated (<50%),5507
